In [2]:
import yaml
import numpy as np
import cv2
from imutils.object_detection import non_max_suppression

def perform_classification(video_src, cascade_src):
    cap = cv2.VideoCapture(video_src)
    car_cascade = cv2.CascadeClassifier(cascade_src)
    while True:
        ret, img = cap.read()
        if (type(img) == type(None)):
            print('Video-ul nu este gasit')
            break
        image_scaled = cv2.resize(img, None, fx=0.6, fy=0.6)
        gray = cv2.cvtColor(image_scaled, cv2.COLOR_BGR2GRAY)
        cars = car_cascade.detectMultiScale(gray, 1.1, 1) #1.1, 1
        cars = np.array([[x, y, x + w, y + h] for (x, y, w, h) in cars])
        pick = non_max_suppression(cars, probs=None, overlapThresh=0.65)
        for (x, y, w, h) in pick:
            # cv2.rectangle(image_scaled, (x, y), (x + w, y + h), (0 , 255, 255), 2) #bgr
            cv2.rectangle(image_scaled, (x, y), (w,  h), (0, 255, 255), 2)
        cv2.imshow('Apasa ESC pentru a iesi din PROGRAM', image_scaled)

            # esc pt iesire
        if cv2.waitKey(33) == 27:
            break
    print('Executie terminata')
    cv2.destroyAllWindows()
    

perform_classification('videos/testvideo_01.avi', 'classifiers/classifier_03.xml')    # autostrada
perform_classification('videos/testvideo_01.avi', 'classifiers/classifier_02.xml')    # autostrada



Executie terminata
Executie terminata


In [1]:
# acuratete classifier
import test_acc as acc
print("Classifier 2 - Utilizat")
acc.get_acc('classifiers/classifier_02.xml')
print("Classifier 1")
acc.get_acc('classifiers/classifier_01.xml')


Classifier 2 - Utilizat
23 autovehicule detectate.
23 autovehicule detectate.
23 autovehicule detectate.
23 autovehicule detectate.
23 autovehicule detectate.
Acuratețe vehicule:  76.66666666666667
Classifier 1
15 autovehicule detectate.
15 autovehicule detectate.
15 autovehicule detectate.
15 autovehicule detectate.
15 autovehicule detectate.
Acuratețe vehicule:  50.0


50.0

In [4]:

import statistics
import cv2
import yaml
import numpy as np
        
sum_up = 0.0
delta_list = []
frame = cv2.imread('images/frame_01.png') #
parking_bounding_rects = []
parking_mask = []
frame_blur = cv2.GaussianBlur(frame.copy(), (5,5), 3)
frame_gray = cv2.cvtColor(frame_blur, cv2.COLOR_BGR2GRAY)
with open('ymls/test_yml_01.yml', 'r') as stream:
    parking_data = yaml.load(stream)
    
if parking_data != None:
    for park in parking_data:
        points = np.array(park['points'])
        rect = cv2.boundingRect(points)
        points_shifted = points.copy()
        points_shifted[:,0] = points[:,0] - rect[0] # schimba conturul
        points_shifted[:,1] = points[:,1] - rect[1]
        
        parking_bounding_rects.append(rect)
        mask = cv2.drawContours(np.zeros((rect[3], rect[2]), dtype=np.uint8), [points_shifted], contourIdx=-1,
                                    color=255, thickness=-1, lineType=cv2.LINE_8)
        mask = mask==255
        parking_mask.append(mask)

for ind, park in enumerate(parking_data):
        points = np.array(park['points'])
        rect = parking_bounding_rects[ind]
        roi_gray = frame_gray[rect[1]:(rect[1]+rect[3]), rect[0]:(rect[0]+rect[2])] # crop roi

        laplacian = cv2.Laplacian(roi_gray, cv2.CV_64F)
        points[:,0] = points[:,0] - rect[0] # schimba conturul roi
        points[:,1] = points[:,1] - rect[1]
        delta = np.mean(np.abs(laplacian * parking_mask[ind]))
        if(delta > 1.8):    # ignora spatiile goale
            delta_list.append(delta)
            # print(delta)
        sum_up = sum_up + delta
        
avg = sum_up/len(parking_data)
med = statistics.median(delta_list)
print("mean: ", avg)
print("median: ", med)

mean:  2.470465407669543
median:  2.3658536585365852


C:\Users\alexm\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:14: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  
